In [1]:
! pip install dash   
! pip install dash-html-components                                         
! pip install dash-core-components                                     
! pip install plotly

In [10]:
import dash
import dash_html_components as html
import plotly.graph_objects as go
import dash_core_components as dcc
from dash.dependencies import Input, Output
from utils import get_modification_count_per_protein
import pandas as pd

path = r"UHT milk P036.csv"
 
df = pd.read_csv(path)

def count_no_of_modifications(ptm_str):
    #check if NaN value
    if pd.isnull(ptm_str):
        return 0
    return 1 + ptm_str.count(';')

#apply count_no_of_modifications to each PTM column
df['#modifications'] = df['PTM'].apply(count_no_of_modifications)
#print non zero values in modifcations column
df[df['#modifications'] > 0]

app = dash.Dash()

colorscales = px.colors.named_colorscales()

def df_to_plotly(df):
    return {'z': df.values.tolist(),
            'x': df.columns.tolist(),
            'y': df.index.tolist()}

app.layout = html.Div(id = 'parent', children = [
    html.H1(id = 'H1', children = 'Distribution of modifications over different proteins', style = {'textAlign':'center',\
                                            'marginTop':40,'marginBottom':40}),
        dcc.Dropdown( id = 'norm_dropdown',
        options = [
            {'label':'Normalization: nothing', "value":"" },
            {'label': 'Normalization: total protein modification count', 'value':"protein_total_mod_count"},
            {'label': 'Normalizaton: protein length', 'value':"protein_length"},
            {'label': 'Normalization: protein mass', 'value':"protein_mass"},
            ],
        value = ''),
        dcc.Dropdown( id = 'count_dropdown',
        options = [
            {'label':'Only include proteins with modification count OVER 10', "value":10 },
            {'label': 'Only include proteins with modification count OVER 20', 'value':20},
            {'label': 'Only include proteins with modification count OVER 40', 'value':40},
            {'label': 'Only include proteins with modification count OVER 80', 'value':80},
            {'label': 'Only include proteins with modification count OVER 160', 'value':160},
            {'label': 'Only include proteins with modification count OVER 320', 'value':320},
            {'label': 'Only include proteins with modification count OVER 640', 'value':640},
            ],
        value = 80),
        dcc.Dropdown(
        id='colorscale', 
        options=[{"value": x, "label": x} 
                 for x in colorscales],
        value='viridis'
        ),
        dcc.Graph(id = 'mod_heatmap_plot'),
                dcc.Loading(
            id="loading-1",
            type="default",
            children=html.Div(id="loading-output-1")
        ),
    ])
    
    
@app.callback(Output(component_id='mod_heatmap_plot', component_property= 'figure'),
              [Input(component_id='norm_dropdown', component_property= 'value'), Input(component_id='count_dropdown', component_property= 'value'), Input("colorscale", "value")])
def graph_update(dropdown_value_norm, dropdown_value_count, scale):
    print(dropdown_value_norm)
    print(dropdown_value_count)
    modPd = pd.DataFrame(get_modification_count_per_protein(df, dropdown_value_count, '{0}'.format(dropdown_value_norm)))
    fig = go.Figure(data=go.Heatmap(df_to_plotly(modPd), colorscale = scale))
    fig.update_layout(title = 'Distribution of modifications over proteins',
                      xaxis_title = 'Name of protein',
                      yaxis_title = 'Name of modification'
                      )
    return fig  


app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Oct/2021 14:58:51] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Oct/2021 14:58:51] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_0_0m1633432702.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [05/Oct/2021 14:58:51] "GET /_dash-component-suites/dash/dcc/dash_core_components.v2_0_0m1633432702.js HTTP/1.1" 200 -
127.0.0.1 - - [05/Oct/2021 14:58:51] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_0_0m1633432702.min.js HTTP/1.1" 200 -
127.0.0.1 - - [05/Oct/2021 14:58:51] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_0_0m1633432702.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [05/Oct/2021 14:58:51] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_0_0m1633432702.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [05/Oct/2021 14:58:51] "GET /_dash-component-suites/dash/dcc/dash_core_components-shared.v2_0_0m1633432702.js HTTP/1.1" 200 -
127.0.0.1 - - [05/Oct/2021 14:58:51] "GET /_dash-component


80
normalization: 
norm is: no normalization


127.0.0.1 - - [05/Oct/2021 14:58:55] "POST /_dash-update-component HTTP/1.1" 200 -



80
normalization: 
norm is: no normalization


...

127.0.0.1 - - [05/Oct/2021 15:42:40] "POST /_dash-update-component HTTP/1.1" 200 -



10
normalization: 
norm is: no normalization
protein_mass
10
normalization: protein_mass
norm is: protein_mass
protein_mass
10
normalization: protein_mass
norm is: protein_mass
protein_mass
320
normalization: protein_mass
norm is: protein_mass


127.0.0.1 - - [05/Oct/2021 15:42:59] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Oct/2021 15:43:00] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Oct/2021 15:43:02] "POST /_dash-update-component HTTP/1.1" 200 -


protein_mass
320
normalization: protein_mass
norm is: protein_mass


127.0.0.1 - - [05/Oct/2021 15:46:21] "POST /_dash-update-component HTTP/1.1" 200 -
